In [10]:
# This file contains functions to create an .out file containing predictions
# Receives a model .h5 file 
# Loop among test files, these files have 8 timesteps and 12 empty timesteps to be filled
# Send 8 times steps to model.predict() and receives 12 predicted future timesteps
# Fill the empty slots
# Write an output file

In [1]:
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# path variables

base_path = "/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/"
model_dir = base_path+"results/"+"20201017-003617"
model_path = model_dir+"/biLSTM"

In [4]:
# Load model

model = tf.keras.models.load_model(model_path)

In [ ]:
def to_relative_coord(dataframe):

    base = dataframe.iloc[0]

    for i in range(0, dataframe.shape[0]-1,1):
        
        if(i % 20 == 0):
            base= dataframe.iloc[i]

        dataframe.at[i, 'x'] = dataframe.at[i, 'x']-base.x
        dataframe.at[i, 'y'] = dataframe.at[i, 'y']-base.y

        

    return dataframe

In [ ]:
def to_absolute_coord(dataframe, dataframe_relative):

    base = dataframe.iloc[0]

    for i in range(0, dataframe.shape[0]-1,1):
        
        if(i % 20 == 0):
            base= dataframe.iloc[i]

        dataframe_relative.at[i, 'x'] = dataframe_relative.at[i, 'x']+base.x
        dataframe_relative.at[i, 'y'] = dataframe_relative.at[i, 'y']+base.y

        

    return dataframe_relative

In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pickle as pkl

# Read and concat test datasets

In [6]:
test_files = [r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/coupa_0.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/coupa_1.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/gates_2.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/hyang_0.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/hyang_1.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/hyang_3.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/hyang_8.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/little_0.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/little_1.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/little_2.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/little_3.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/nexus_5.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/nexus_6.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/quad_0.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/quad_1.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/quad_2.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/stanford/quad_3.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/crowds/crowds_zara01.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/crowds/uni_examples.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction/data/test/biwi/biwi_eth.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/stanford/bookstore_0.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/biwi_hotel.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/crowds_zara02.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/deathCircle_0.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/PETS09-S2L1.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/students001.txt',
             r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/stanford/bookstore_3.txt']

In [ ]:
# only one test file, comment this line to generate all test output files
#test_files = [r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/stanford/bookstore_0.txt',
#              r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/biwi_hotel.txt',
#              r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/crowds_zara02.txt',
#              r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/deathCircle_0.txt',
#              r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/PETS09-S2L1.txt',
#              r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/students001.txt',
#              r'/content/drive/My Drive/RESEARCH/2020/Pedestrian_prediction_simposio/data/local-test/stanford/bookstore_3.txt']

In [7]:
n_steps = 8;
n_features = 1;

empty_values=['?']

for current_file in test_files:
    
    dataframe = pd.DataFrame()

    dataframe = pd.read_csv(current_file, sep=' ',header=None, names = ['time','id', 'x', 'y'], na_values=empty_values)
    dataframe.id = dataframe.id.astype(int)
	
    # convert x,y sequences, using x,y values relative to first step, only used
    # if the model was trained with relative values
    #dataframe_absolute = dataframe.copy()
    #dataframe = to_relative_coord(dataframe)


    # min max scaling
    # normalize all columns but not id column
    id_column = dataframe['id']
    time_column = dataframe['time']
    dataframe.drop(columns=['id','time'], axis=1, inplace=True) # we dont use the time column for trajectory prediction

    scaler = MinMaxScaler()
    # Load scaler that was fitted on training data
    with open(model_dir+"/train_scaler.pkl", "rb") as infile:
        scaler = pkl.load(infile)
    
    dataframe = pd.DataFrame(scaler.transform(dataframe), columns=dataframe.columns)

    dataframe = dataframe.join(id_column)   # re insert the id column to the scaled dataframe
    dataframe = dataframe.join(time_column) # re insert the time column to the scaled dataframe

    # split the x and y features for multiple input model
    dataframe_x = dataframe.drop(columns=['y'], axis=1)
    dataframe_y = dataframe.drop(columns=['x'], axis=1)
    

    test_rows = dataframe.shape[0]

# this loop fills the steps using sliding window method

#    for i in range(0,test_rows-19,20):
#        for j in range(i,i+12):
#            x_input = (dataframe.iloc[j:j+8,0:2]).values
#            x_input = x_input.reshape((1, n_steps, n_features))
            
            #x_input = x_input.reshape(1,16)  # 16 = 8 steps x 2 features for use with MLP

#            yhat = model.predict(x_input,verbose=0)    
    
            # single step fill
#            dataframe.iloc[j+8, dataframe.columns.get_loc('x')] = yhat[0][0]
#            dataframe.iloc[j+8, dataframe.columns.get_loc('y')] = yhat[1][0]

# this loop fills the steps using multioutput prediction. 12 steps prediction

    for i in range(0,test_rows-19,20):
#        for j in range(i,i+12):
        x1_input = (dataframe_x.iloc[i:i+8,0:1]).values
        x1_input = x1_input.reshape((1, n_steps, n_features))

        x2_input = (dataframe_y.iloc[i:i+8,0:1]).values
        x2_input = x2_input.reshape((1, n_steps, n_features))
            
            #x_input = x_input.reshape(1,16)  # 16 = 8 steps x 2 features for use with MLP

        yhat = model.predict([x1_input,x2_input],verbose=0)    
        #print(yhat.shape)
        #print(yhat[0].shape)
        #print(yhat[0])
        #print(yhat[0][0])
        #print(yhat[0][0][0])
        #print(yhat[0][0][1])
        
        #dataframe.iloc[i+8, dataframe.columns.get_loc('x')] = yhat[0][0][0]
        #dataframe.iloc[i+8, dataframe.columns.get_loc('y')] = yhat[0][0][1]

        for j in range(0,12):
            #print(str(yhat[0][j][0])+" "+str(yhat[0][j][1]))
            dataframe.iloc[i+8+j, dataframe.columns.get_loc('x')] = yhat[0][j][0] 
            dataframe.iloc[i+8+j, dataframe.columns.get_loc('y')] = yhat[0][j][1]

           
            
    

    id_column = dataframe['id']
    time_column = dataframe['time']
    dataframe.drop(columns=['id','time'], axis=1, inplace=True) # we dont use the time column for trajectory prediction
    dataframe = pd.DataFrame(scaler.inverse_transform(dataframe), columns=dataframe.columns)
    dataframe = dataframe.join(id_column)   # re insert the id column to the scaled dataframe
    dataframe = dataframe.join(time_column) # re insert the time column to the scaled dataframe

    #dataframe.x = dataframe.x.round(3)
    #dataframe.y = dataframe.y.round(3)

    # revert to absolute coordinates
    #dataframe.x = dataframe.x + dataframe_absolute.x
    #dataframe.y = dataframe.y + dataframe_absolute.y
    #dataframe = to_absolute_coord(dataframe_absolute, dataframe)

    dataframe.x = dataframe.x.round(3)
    dataframe.y = dataframe.y.round(3)
    
    dataframe = dataframe[['time', 'id','x','y']]
    dataframe.to_csv(current_file+".out", header=None, index=None, sep=' ', mode='w')